## Luck vs. Skill (Emerging Markets Edition)

Goal of this notebook is to test out the tools I made in the python file `luck_v_skill.py`, as well as try out some new data.

In [15]:
import sys
sys.path.insert(0, '/Users/danielsalas/Projects/Fama-French-Funds/src')

from luck_v_skill import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [68]:
# Load data from local files
funds = pd.read_csv('../data/EM Funds.csv',parse_dates=['Dates'],index_col='Dates')
factors = pd.read_csv('../data/EM Factors.csv',parse_dates=['Dates'],index_col='Dates')

funds.shape, factors.shape

((214, 812), (214, 9))

In [70]:
# Start counting how long this program takes
start_time = time.time()

# select columns of interest
factor_cols = ['EMRF','HML','SMB','RF'] # careful picking factors...multicollinearity is not fun
factors = factors[factor_cols]

# initialize instance of the AlphaEvaluator with fund and factor data
AEV = AlphaEvaluator(fund_data=funds,factor_data=factors)

AEV.fit(min_obs=120)
AEV.simulate(n_simulations=1000,random_seed=2,sim_std=0,sim_cutoff=15,verbose=True)
AEV.get_percentiles()
data_a, data_t = AEV.data_a.copy(), AEV.data_t.copy()

# Rename data columns
data_a.columns = ['Actual','Sim Avg (0.00)', '%<Act (0.00)']
data_t.columns = ['Actual','Sim Avg (0.00)', '%<Act (0.00)']

for i,stdev in enumerate(np.arange(1,4)/4):
    title_print("Standard Deviation {} of 3".format(i+1))
    # Rerun simulation with new standard deviation injection for alpha
    AEV.simulate(n_simulations=1000,random_seed=2,sim_std=stdev,sim_cutoff=15,verbose=True)
    
    # Update Percentile Data
    print("Getting percentiles")
    AEV.get_percentiles()
    
    # Alphas
    data_a['Sim Avg ({:.2f})'.format(stdev)] = AEV.data_a['Sim Avg']
    data_a['%<Act ({:.2f})'.format(stdev)] = AEV.data_a['%<Act']
    
    # t-statistics for Alpha
    data_t['Sim Avg ({:.2f})'.format(stdev)] = AEV.data_t['Sim Avg']
    data_t['%<Act ({:.2f})'.format(stdev)] = AEV.data_t['%<Act']

print("DONE!")
t_diff = time.time()-start_time
minutes = int(np.floor(t_diff/60))
seconds = t_diff - 60*minutes
print("{} minutes and {:.2f} seconds elapsed for this program".format(minutes,seconds))

Annual standard deviation: 0.00, Standard deviation alpha: 0.00
Starting 206,000 regressions...
Simulations complete in 89.21 seconds!
-------------------------
Standard Deviation 1 of 3
-------------------------
Annual standard deviation: 0.25, Standard deviation alpha: 0.07
Starting 206,000 regressions...
Simulations complete in 86.17 seconds!
Getting percentiles
-------------------------
Standard Deviation 2 of 3
-------------------------
Annual standard deviation: 0.50, Standard deviation alpha: 0.14
Starting 206,000 regressions...
Simulations complete in 88.59 seconds!
Getting percentiles
-------------------------
Standard Deviation 3 of 3
-------------------------
Annual standard deviation: 0.75, Standard deviation alpha: 0.22
Starting 206,000 regressions...
Simulations complete in 90.19 seconds!
Getting percentiles
DONE!
5 minutes and 54.82 seconds elapsed for this program


In [71]:
data_a

,Actual,Sim Avg (0.00),%<Act (0.00),Sim Avg (0.25),%<Act (0.25),Sim Avg (0.50),%<Act (0.50),Sim Avg (0.75),%<Act (0.75)
Worst,-0.305305,-0.390164,67.9,-0.490703,89.4,-0.700094,99.4,-0.950316,100.0
2nd,-0.304176,-0.299710,43.3,-0.382338,74.9,-0.544159,97.9,-0.738598,100.0
3rd,-0.286605,-0.262063,35.3,-0.333514,65.4,-0.475089,97.6,-0.641153,100.0
4th,-0.273418,-0.240474,30.7,-0.301092,56.3,-0.428395,96.1,-0.582255,99.9
5th,-0.260319,-0.226434,30.0,-0.277594,53.3,-0.395297,95.0,-0.536283,99.9
10%,-0.182447,-0.260983,78.6,-0.331892,96.0,-0.472755,99.9,-0.638208,100.0
20%,-0.116897,-0.224690,91.8,-0.275691,98.6,-0.392742,100.0,-0.532879,100.0
30%,-0.075017,-0.193880,96.8,-0.241498,99.4,-0.346132,100.0,-0.469659,100.0
40%,-0.045551,-0.174706,98.4,-0.218145,99.6,-0.312754,100.0,-0.425220,100.0
50%,-0.021168,-0.159706,98.9,-0.200448,99.8,-0.287465,100.0,-0.391796,100.0


In [72]:
data_t

,Actual,Sim Avg (0.00),%<Act (0.00),Sim Avg (0.25),%<Act (0.25),Sim Avg (0.50),%<Act (0.50),Sim Avg (0.75),%<Act (0.75)
Worst,-6.145405,-3.407499,1.3,-4.555164,8.7,-6.660503,62.0,-9.090764,96.5
2nd,-5.619018,-3.014192,1.4,-3.891836,4.4,-5.694110,49.5,-7.816018,95.1
3rd,-5.218572,-2.793097,1.0,-3.566099,4.9,-5.205490,47.1,-7.116694,95.4
4th,-4.622620,-2.633254,2.7,-3.353521,8.1,-4.875998,57.2,-6.669352,97.5
5th,-4.470173,-2.496557,2.7,-3.179958,7.2,-4.618615,53.3,-6.340773,96.4
10%,-2.510529,-2.785105,59.0,-3.555470,88.1,-5.189016,99.8,-7.094327,100.0
20%,-1.724592,-2.485407,80.3,-3.165171,96.7,-4.597763,100.0,-6.312455,100.0
30%,-1.283830,-2.271038,88.1,-2.893905,97.8,-4.215457,100.0,-5.775760,100.0
40%,-0.822617,-2.116219,95.6,-2.688326,99.4,-3.912170,100.0,-5.378550,100.0
50%,-0.411077,-1.994937,98.7,-2.524163,99.7,-3.673082,100.0,-5.039942,100.0


In [73]:
data_a.to_csv('../EM-luck-vs-skill-alphas.csv')
data_t.to_csv('../EM-luck-vs-skill-tstats.csv')